## Navigators
Navigators provide the ability to drill into the result set providing a summary of the distribution.  They are particularly useful when you are interested in gathering the domain of values for a particular Property.  Navigators can be used against a specific View, used in conjunction with either a query, a filter or both.  Navigators can be simple or very powerful, but provide a very useful way to capture results in logical buckets. 

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.DataPlatform.Content, 1.0.0-alpha3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Refinitiv.DataPlatform.Content version 1.0.0-alpha3

Installed package Microsoft.Data.Analysis version 0.4.0

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.DataPlatform.Content.SearchService;
using Refinitiv.DataPlatform.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Linq;
using System;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<IList<JObject>>((hits, writer) =>
{
    if (hits.Count > 0)
    {
        var fields = new List<String>();
        var rows = new List<ICollection<IHtmlContent>>();
        var data = new Dictionary<string, IHtmlContent>();

        foreach(var hit in hits)
        {
            var cells = new List<IHtmlContent>();
            foreach (var val in hit.Properties())
            {
                if ( !fields.Any(item => item.Equals(val.Name)) )
                    fields.Add(val.Name);
                data[val.Name] = td(val.Value.ToString());
            }
            rows.Add(new List<IHtmlContent>(data.Values));
            data.Keys.ToList().ForEach(x => data[x] = td(""));
        }
        
        var headers = new List<IHtmlContent>();
        headers.AddRange(fields.Select(c => (IHtmlContent)th(c)));

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
var session = CoreFactory.CreateSession(new DesktopSession.Params()
                            .AppKey("Your API Key here")
                            .OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                            .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})")));
session.Open();

17/12/2020 12:23:29 PM:Session is Pending. (State: Pending)
17/12/2020 12:23:30 PM:{
  "Contents": "Desktop Session Successfully Authenticated"
}. (Event: SessionAuthenticationSuccess)
17/12/2020 12:23:30 PM:Session is Opened. (State: Opened)


#### Example - Property values
The most basic usage of navigators is to determine the available values within a given Property.  For example, when searching for assets, we may want to narrow down the specific country an asset is traded in.  We can determine the available countries to work with.

In [5]:
// The following will list the countries assets are listed.  We can use this output to filter based on a specific country.
var response = Search.Definition(Search.View.SearchAll).Top(0)
                                                       .Navigators("RCSCountryLeaf(buckets:20)")  // Show the top 20
                                                       .GetData();
Console.WriteLine(response.Data.Navigators["RCSCountryLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 11850499
  },
  {
    "Label": "Eurobond Market",
    "Count": 3613299
  },
  {
    "Label": "Colombia",
    "Count": 1266336
  },
  {
    "Label": "China (Mainland)",
    "Count": 1227295
  },
  {
    "Label": "South Korea",
    "Count": 533785
  },
  {
    "Label": "Germany",
    "Count": 469964
  },
  {
    "Label": "Canada",
    "Count": 374106
  },
  {
    "Label": "France",
    "Count": 221276
  },
  {
    "Label": "India",
    "Count": 186144
  },
  {
    "Label": "Japan",
    "Count": 156881
  },
  {
    "Label": "Chile",
    "Count": 152045
  },
  {
    "Label": "Italy",
    "Count": 135890
  },
  {
    "Label": "Australia",
    "Count": 86807
  },
  {
    "Label": "Spain",
    "Count": 58654
  },
  {
    "Label": "Switzerland",
    "Count": 58418
  },
  {
    "Label": "Russia",
    "Count": 56574
  },
  {
    "Label": "Austria",
    "Count": 55442
  },
  {
    "Label": "Norway",
    "Count": 52863
  },
  {
    "Label": "United 

#### Example - Vessels
If I want to perform a basic search for tankers and capture the Hull Types, I could start with the following basic query:

In [6]:
response = Search.Definition(Search.View.VesselPhysicalAssets).Query("tanker")
                                                              .Select("DocumentTitle, HullType")
                                                              .Top(10000)
                                                              .GetData();
response.Data.Hits

"FPSO YUUM KAK NAAB, LNG Tanker, BW OFFSHORE SINGAPORE PTE LTD, Gulf of Mexico|East Mexico Tanker Zone|Mexico",Single Hull
"MOZAH, LNG Tanker, NAKILAT SHIPPING QATAR LTD, East Indian Ocean|Colombo Tanker Zone",Single Hull
"UMM SLAL, LNG Tanker, NAKILAT SHIPPING QATAR LTD, East China Sea|Korea Tanker Zone|South Korea",Single Hull
"BU SAMRA, LNG Tanker, STASCO SHIP MANAGEMENT, Gulf of Oman|Arabian Gulf Tanker Zone",Double Hull
"AL MAYEDA, Liquefied Natural Gas; LNG Tanker, STASCO SHIP MANAGEMENT, Gulf of Oman|Arabian Gulf Tanker Zone",Double Hull
"MEKAINES, LNG Tanker, NAKILAT SHIPPING QATAR LTD, West Indian Ocean|Colombo Tanker Zone",Single Hull
"AL MAFYAR, Liquefied Natural Gas; LNG Tanker, NAKILAT SHIPPING QATAR LTD, East China Sea|Taiwan Tanker Zone",Single Hull
"SHAGRA, LNG Tanker, QATARGAS LIQUEFIED GAS CO LTD, Arabia Sea",Double Hull
"ZARGA, Liquefied Natural Gas; LNG Tanker, QATAR GAS TRANSPORT - NAKILAT, East Indian Ocean",Single Hull
"AAMIRA, Liquefied Natural Gas; LNG Tanker, STASCO SHIP MANAGEMENT, Java Sea|Singapore Tanker Zone|Singapore Strait Waypoint",Single Hull
"RASHEEDA, LNG Tanker, QATARGAS LIQUEFIED GAS CO LTD, Java Sea|Singapore Tanker Zone|Singapore Strait Waypoint|Singapore",Single Hull


Here, we can see a list of the different tankers and their Hull Types. Within this collection, while I can clearly see there are 2 different Hull Types: "Single" and "Double", I'm also seeing duplicates.  Instead of going through the effort to process the above result to pull out the unique list of Hull Types, I will instead use a Navigator - which was designed to do this work for me.

Using the above query, I will apply a Navigator by selecting the 'HullType' Property.  What this does is instruct search to bucket all unique Hull Types and summarize the total found for each. To capture the results, I will need to utilize the Content Layer API call.

In [7]:
response = Search.Definition(Search.View.VesselPhysicalAssets).Query("tanker")
                                                              .Top(0)  // Only interested in Navigator details
                                                              .Navigators("HullType")
                                                              .GetData();
Console.WriteLine(response.Data.Navigators["HullType"]["Buckets"]);

[
  {
    "Label": "Double Hull",
    "Count": 16882
  },
  {
    "Label": "Single Hull",
    "Count": 5131
  }
]


#### Example - Listing of exchanges per asset category
In the following example, I'm going to list the top 3 asset types in Canada and for each, display the top 2 exchanges trading on those assets.

In [8]:
// Filter on all active (AC) assets within Canada
var filter = "AssetState eq 'AC' and RCSExchangeCountryLeaf eq 'Canada'";

// For the top 3 categories, show the top 2 exchanges
var navigators = "RCSAssetCategoryLeaf(buckets:3, sub:ExchangeName(buckets:2))"; 

response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Navigators(navigators)
                                                      .Top(0)
                                                      .GetData();

In [9]:
// Pretty display of the listing
var rank = 1;
var exchangeRank = 1;
foreach (var asset in response.Data.Navigators["RCSAssetCategoryLeaf"]["Buckets"])
{
    Console.WriteLine($"{rank++}. {asset["Label"]}");
    foreach (var exchange in asset["ExchangeName"]["Buckets"])
        Console.WriteLine($"\t{exchangeRank++}. {exchange["Label"]} ({exchange["Count"]})");
    exchangeRank = 1;
}

1. Equity Cash Option
	1. Montreal Options Exchange (44788)
	2. Montreal Exchange (26)
2. Ordinary Share
	1. Aequitas NEO-L (3247)
	2. Aequitas NEO-N (3246)
3. Equity Future
	1. Montreal Exchange (7891)
	2. Montreal Options Exchange (109)


#### Example - Outstanding Government debt in the last 3 months.
In this example, I will be using some nested expressions to summarize categories within the buckets I collect.  Specifically, I'm looking for government debt ('govt') issued any time after 3-months ago using the following filter:
- "DBType eq 'govt' and IssueDate gt today-3months" (DBType identifies government debt)
    
Based on the result set, I want categorize or create buckets (top 5) based on the domicile of the issuer.  For each domicile, sum the outstanding debt and also include the maximum coupon rate for each one.
- RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)

In [10]:
// Get date N months ago
var date = DateTime.Now.AddMonths(-3);  // Last 3 months

In [11]:
// Retrieve the top 5 outstanding government debt in the last 3 months
filter = $"DbType eq 'govt' and IssueDate gt {date:yyyy-MM-dd}";
navigators = "RCSDomicileLeaf(buckets:5, desc:sum_FaceOutstandingUSD, calc:max_CouponRate)";

response = Search.Definition(Search.View.GovCorpInstruments).Filter(filter)
                                                            .Navigators(navigators)
                                                            .Top(0)
                                                            .GetData();
Console.WriteLine(response.Data.Navigators["RCSDomicileLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 66,
    "max_CouponRate": 1.625,
    "sum_FaceOutstandingUSD": 2268810302300.0
  },
  {
    "Label": "Japan",
    "Count": 53,
    "max_CouponRate": 0.6,
    "sum_FaceOutstandingUSD": 1715714645993.0
  },
  {
    "Label": "China (Mainland)",
    "Count": 44,
    "max_CouponRate": 3.97,
    "sum_FaceOutstandingUSD": 241468462819.0
  },
  {
    "Label": "Hong Kong",
    "Count": 40,
    "max_CouponRate": 3.5,
    "sum_FaceOutstandingUSD": 139846852522.0
  },
  {
    "Label": "Singapore",
    "Count": 46,
    "max_CouponRate": 0.5,
    "sum_FaceOutstandingUSD": 122182308344.0
  }
]
